<a href="https://colab.research.google.com/github/JessemanGray/GraphUp/blob/main/GraphUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install meshio[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.2/166.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00


In [2]:
!pip install open3d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [4]:
# Import plotly for visualizations
import plotly
plotly.__version__

'5.15.0'

In [3]:
# Imports for graphing and labeling
import os
import numpy as np
import meshio
import plotly.graph_objects as go
import open3d as o3d


In [5]:
# Import ply files from github repository

file_urls = ['https://raw.githubusercontent.com/JessemanGray/GraphUp/main/liberty_hall.ply', 'https://raw.githubusercontent.com/JessemanGray/GraphUp/main/mechanical_floor_1.ply', 'https://raw.githubusercontent.com/JessemanGray/GraphUp/main/electrical_floor_1.ply', 'https://raw.githubusercontent.com/JessemanGray/GraphUp/main/plumbing_floor_1.ply']

for url in file_urls:
  !wget {url}

--2024-02-13 22:54:05--  https://raw.githubusercontent.com/JessemanGray/GraphUp/main/liberty_hall.ply
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 392683 (383K) [application/octet-stream]
Saving to: ‘liberty_hall.ply’

liberty_hall.ply    100%[===================>] 383.48K  --.-KB/s    in 0.03s   

2024-02-13 22:54:06 (14.7 MB/s) - ‘liberty_hall.ply’ saved [392683/392683]

--2024-02-13 22:54:06--  https://raw.githubusercontent.com/JessemanGray/GraphUp/main/mechanical_floor_1.ply
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7

In [7]:
# Assign labels to ply files

model_path = 'liberty_hall.ply'
mech_path = 'mechanical_floor_1.ply'
elec_path = 'electrical_floor_1.ply'
plumb_path = 'plumbing_floor_1.ply'

In [8]:
assert os.path.exists(model_path), 'file does not exist'

In [11]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import open3d as o3d
import numpy as np

# Function to read .ply file and return vertices and triangles
def get_mesh_data(filepath):
    mesh = o3d.io.read_triangle_mesh(filepath)
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    return vertices, triangles

# Function to create Mesh3d object
def create_mesh3d(vertices, triangles, color, opacity):
    return go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        color=color,
        opacity=opacity,
        name=color  # Name the trace with its color for the legend
    )

# Read .ply files for base model and overlays
base_vertices, base_triangles = get_mesh_data(model_path)
mech_vertices, mech_triangles = get_mesh_data(mech_path)
elec_vertices, elec_triangles = get_mesh_data(elec_path)
plumb_vertices, plumb_triangles = get_mesh_data(plumb_path)

'''# Define scaling factor for height
height_scale = 0.0005 # Adjust as needed

# Scale height of base model and overlay meshes
base_vertices[:, 2] *= height_scale
mech_vertices[:, 2] *= height_scale  # corrected from mech_vertices[:, 1]
elec_vertices[:, 2] *= height_scale  # corrected from elec_vertices[:, 1]
plumb_vertices[:, 2] *= height_scale  # corrected from plumb_vertices[:, 1]

# Define translation factor for height
height_translation = 0.000001  # Adjust as needed

# Translate height of overlay meshes
mech_vertices[:, 2] += height_translation
elec_vertices[:, 2] += height_translation * .5 # Multiply by 2 to create a gap between layers
plumb_vertices[:, 2] += height_translation * .75  # Multiply by 3 to create a gap between layers

# Define new range for z-coordinates
z_min_new = 0.0  # Adjust as needed
z_max_new = 0.0000001  # Adjust as needed

# Normalize z-coordinates of overlay meshes
mech_vertices[:, 2] = z_min_new + (mech_vertices[:, 2] - np.min(mech_vertices[:, 2])) * (z_max_new - z_min_new) / (np.max(mech_vertices[:, 2]) - np.min(mech_vertices[:, 2]))
elec_vertices[:, 2] = z_min_new + (elec_vertices[:, 2] - np.min(elec_vertices[:, 2])) * (z_max_new - z_min_new) / (np.max(elec_vertices[:, 2]) - np.min(elec_vertices[:, 2]))
plumb_vertices[:, 2] = z_min_new + (plumb_vertices[:, 2] - np.min(plumb_vertices[:, 2])) * (z_max_new - z_min_new) / (np.max(plumb_vertices[:, 2]) - np.min(plumb_vertices[:, 2]))

# Get maximum z-coordinate of base layer
z_max_base = np.max(base_vertices[:, 2])

# Adjust minimum z-coordinate of overlay meshes to match maximum z-coordinate of base layer
mech_vertices[:, 2] -= np.min(mech_vertices[:, 2]) - z_max_base
elec_vertices[:, 2] -= np.min(elec_vertices[:, 2]) - z_max_base
plumb_vertices[:, 2] -= np.min(plumb_vertices[:, 2]) - z_max_base

# Define scaling factor for base layer height
height_scale_base = 0.00015  # Decrease this number to reduce the height of the base layer

# Scale height of base layer
base_vertices[:, 2] *= height_scale_base'''


# Define Dash app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='3d-plot'),
    html.Label('Mechanical Opacity Slider'),
    dcc.Slider(id='mech-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Electrical Opacity Slider'),
    dcc.Slider(id='elec-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Plumbing Opacity Slider'),
    dcc.Slider(id='plumb-opacity-slider', min=0, max=1, step=0.1, value=0.5),
])

@app.callback(
    Output('3d-plot', 'figure'),
    [Input('mech-opacity-slider', 'value'),
     Input('elec-opacity-slider', 'value'),
     Input('plumb-opacity-slider', 'value')]
)
def update_graph(mech_opacity, elec_opacity, plumb_opacity):
    fig = go.Figure(
        data=[
            go.Mesh3d(
                # Base plot
                x=base_vertices[:, 0],
                y=base_vertices[:, 1],
                z=base_vertices[:, 2],
                i=base_triangles[:, 0],
                j=base_triangles[:, 1],
                k=base_triangles[:, 2],
                color='lightyellow',
                opacity=0.5
            ),
            go.Mesh3d(
                # Mechanical overlay
                x=mech_vertices[:, 0],
                y=mech_vertices[:, 1],
                z=mech_vertices[:, 2],
                i=mech_triangles[:, 0],
                j=mech_triangles[:, 1],
                k=mech_triangles[:, 2],
                color='orange',
                opacity=mech_opacity,  # Controlled by the slider
                name='Mechanical'
            ),
            go.Mesh3d(
                # Electrical overlay
                x=elec_vertices[:, 0],
                y=elec_vertices[:, 1],
                z=elec_vertices[:, 2],
                i=elec_triangles[:, 0],
                j=elec_triangles[:, 1],
                k=elec_triangles[:, 2],
                color='hotpink',
                opacity=elec_opacity,  # Controlled by the slider
                name='Electrical'
            ),
            go.Mesh3d(
                # Plumbing overlay
                x=plumb_vertices[:, 0],
                y=plumb_vertices[:, 1],
                z=plumb_vertices[:, 2],
                i=plumb_triangles[:, 0],
                j=plumb_triangles[:, 1],
                k=plumb_triangles[:, 2],
                color='turquoise',
                opacity=plumb_opacity,  # Controlled by the slider
                name='Plumbing'
            ),
        ],
        layout=go.Layout(
            autosize=False,
            width=800,
            height=400,
            plot_bgcolor='black',
            paper_bgcolor='black',
            font=dict(size=16),
            scene=dict(
                xaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False),
                zaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False)
            )
        )
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>